<a href="https://colab.research.google.com/github/dxvsh/LearningPytorch/blob/main/Week2/DLP_GA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DLP GA2

In [ ]:
from pprint import pprint

In [ ]:
!pip install datasets > /dev/null

In [5]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import Lowercase

Download the BookCorpus dataset. Take every 7-th sample (the indices are multiple of 7:[0,7,14,21,...]) from the entire dataset. This will result in a dataset with 10 million samples (exactly, 10,572,033). Use these samples to build a tokenizer with the BPE tokenization algorithm by varying the vocabulary size.

- **Normalizer**: LowerCase
- **PreTokenizer**: WhiteSpace
- **Model**: `BPE`
- **Special tokens**: `[GO],[UNK],[PAD],[EOS]`
- **PostProcessing**: None


Tokenize the input text: “SEBI study finds 93% of individual F&O traders made losses between FY22 and FY24.” using the following configurations.

In [4]:
ds = load_dataset("bookcorpus", split="train")

bookcorpus.py:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

The repository for bookcorpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bookcorpus.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

In [6]:
new_ds = ds.select(range(0, len(ds), 7))  # take every 7-th sample

In [7]:
len(new_ds)

10572033

In [8]:
def get_examples(batch_size=1000):
    for i in range(0, len(new_ds), batch_size):
        yield new_ds[i : i + batch_size]["text"]

In [13]:
input_text = "SEBI study finds 93% of individual F&O traders made losses between FY22 and FY24."

**Q1**. Keep the vocabulary size at 5000 and tokenize the input text using the learned vocabulary. Choose the number of tokens returned by the tokenizer:


- [ ] 18
- [ ] 22
- [ ] 25
- [ ] 28
- [x] 32
- [ ] 60


In [25]:
tokenizer_5k = Tokenizer(BPE(unk_token="[UNK]"))

# set the normalizer and pre-tokenizer
tokenizer_5k.normalizer = Lowercase()
tokenizer_5k.pre_tokenizer = Whitespace()

In [26]:
# setup the trainer

trainer_5k = BpeTrainer(
    vocab_size=5000,
    special_tokens=["[GO]", "[UNK]", "[PAD]", "[EOS]"],
    continuing_subword_prefix="##",
)

In [27]:
tokenizer_5k.train_from_iterator(get_examples(batch_size=5000), trainer=trainer_5k, length=len(new_ds))

In [28]:
tokenizer_5k.get_vocab_size()

5000

In [29]:
tokenizer_5k.encode(input_text)

Encoding(num_tokens=32, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

So with a vocab size of 5K, the tokenizer used 32 tokens for encoding the input text.

In [30]:
pprint(tokenizer_5k.encode(input_text).tokens, compact=True)

['seb', '##i', 'study', 'find', '##s', '9', '##3', '%', 'of', 'ind', '##ivid',
 '##ual', 'f', '&', 'o', 'tr', '##ad', '##ers', 'made', 'loss', '##es',
 'between', 'f', '##y', '##2', '##2', 'and', 'f', '##y', '##2', '##4', '.']


**Q2.** Increase the vocabulary size to 10K, 15K and 32K. For each case, tokenize the same input with the newly learned vocabulary. Choose all the correct statements (MSQ)

- [x] Increasing vocab size from 5K to 10K decreases the number of tokens
- [ ] Increasing vocab size from 5K to 10K increases the number of tokens
- [ ] Increasing vocab size from 10K to 15K decreases the number of tokens
- [x] Increasing vocab size from 10K to 15K does not change the number of tokens
- [x] Increasing vocab size from 15K to 32K further decreases the number of tokens


In [31]:
tokenizer_10k = Tokenizer(BPE(unk_token="[UNK]"))

# set the normalizer and pre-tokenizer
tokenizer_10k.normalizer = Lowercase()
tokenizer_10k.pre_tokenizer = Whitespace()

trainer_10k = BpeTrainer(
    vocab_size=10_000,
    special_tokens=["[GO]", "[UNK]", "[PAD]", "[EOS]"],
    continuing_subword_prefix="##",
)

In [32]:
tokenizer_10k.train_from_iterator(get_examples(batch_size=5000), trainer=trainer_10k, length=len(new_ds))

In [33]:
tokenizer_10k.encode(input_text)

Encoding(num_tokens=28, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

We can observe that increasing the vocab size to 10k, decreased the number of required tokens from 32 to 28.

In [34]:
pprint(tokenizer_10k.encode(input_text).tokens, compact=True)

['seb', '##i', 'study', 'finds', '9', '##3', '%', 'of', 'individual', 'f', '&',
 'o', 'trad', '##ers', 'made', 'loss', '##es', 'between', 'f', '##y', '##2',
 '##2', 'and', 'f', '##y', '##2', '##4', '.']


In [37]:
tokenizer_15k = Tokenizer(BPE(unk_token="[UNK]"))

# set the normalizer and pre-tokenizer
tokenizer_15k.normalizer = Lowercase()
tokenizer_15k.pre_tokenizer = Whitespace()

trainer_15k = BpeTrainer(
    vocab_size=15_000,
    special_tokens=["[GO]", "[UNK]", "[PAD]", "[EOS]"],
    continuing_subword_prefix="##",
)

tokenizer_15k.train_from_iterator(get_examples(batch_size=5000), trainer=trainer_15k, length=len(new_ds))

In [38]:
pprint(tokenizer_15k.encode(input_text))

Encoding(num_tokens=28, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In this case, going from a vocab size of 10k to 15k did not impact the number of tokens used for encoding at all

In [39]:
pprint(tokenizer_15k.encode(input_text).tokens, compact=True)

['seb', '##i', 'study', 'finds', '9', '##3', '%', 'of', 'individual', 'f', '&',
 'o', 'trad', '##ers', 'made', 'loss', '##es', 'between', 'f', '##y', '##2',
 '##2', 'and', 'f', '##y', '##2', '##4', '.']


In [40]:
tokenizer_32k = Tokenizer(BPE(unk_token="[UNK]"))

# set the normalizer and pre-tokenizer
tokenizer_32k.normalizer = Lowercase()
tokenizer_32k.pre_tokenizer = Whitespace()

trainer_32k = BpeTrainer(
    vocab_size=32_000,
    special_tokens=["[GO]", "[UNK]", "[PAD]", "[EOS]"],
    continuing_subword_prefix="##",
)

tokenizer_32k.train_from_iterator(get_examples(batch_size=5000), trainer=trainer_32k, length=len(new_ds))

In [41]:
pprint(tokenizer_32k.encode(input_text))

Encoding(num_tokens=25, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [42]:
pprint(tokenizer_32k.encode(input_text).tokens, compact=True)

['seb', '##i', 'study', 'finds', '9', '##3', '%', 'of', 'individual', 'f', '&',
 'o', 'traders', 'made', 'losses', 'between', 'f', '##y', '##22', 'and', 'f',
 '##y', '##2', '##4', '.']


Observe that going to a vocab size of 32K further reduced the number of tokens required by a little bit, from 28 to 25.

**Q3.** Download the pre-trained tokenizer file “hopper.json” used in the lecture, from [here](https://drive.google.com/file/d/1QNnyh8iMN-IqW_h1w8gAMtw09Em7-e1e/view?usp=sharing) . The tokenizer was trained on all 70 million samples in the BookCorpus dataset. Tokenize the same input text using this “hopper” tokenizer. How many tokens are there?

[After finding the answer, take a moment to compare the hopper tokenizer with the previous one]

In [43]:
# download the given file using wget

!wget 'https://drive.google.com/uc?export=download&id=1QNnyh8iMN-IqW_h1w8gAMtw09Em7-e1e' -O hopper.json

--2024-10-05 15:46:33--  https://drive.google.com/uc?export=download&id=1QNnyh8iMN-IqW_h1w8gAMtw09Em7-e1e
Resolving drive.google.com (drive.google.com)... 142.251.2.102, 142.251.2.101, 142.251.2.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1QNnyh8iMN-IqW_h1w8gAMtw09Em7-e1e&export=download [following]
--2024-10-05 15:46:34--  https://drive.usercontent.google.com/download?id=1QNnyh8iMN-IqW_h1w8gAMtw09Em7-e1e&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.2.132, 2607:f8b0:4023:c0d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1357230 (1.3M) [application/octet-stream]
Saving to: ‘hopper.json’

hopper.json         100%[===================>]   1.29M  --.-KB/s    in 0.

In [59]:
trained_tokenizer = Tokenizer(BPE())
trained_tokenizer = trained_tokenizer.from_file("hopper.json")

In [60]:
trained_tokenizer.encode(input_text)

Encoding(num_tokens=25, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [61]:
pprint(trained_tokenizer.encode(input_text).tokens, compact=True)

['seb', '##i', 'study', 'finds', '9', '##3', '%', 'of', 'individual', 'f', '&',
 'o', 'traders', 'made', 'losses', 'between', 'f', '##y', '##22', 'and', 'f',
 '##y', '##2', '##4', '.']


In [62]:
trained_tokenizer.get_vocab_size()

32000

Using the tokenizer trained on the full 70 million samples still gives us **25 tokens**... maybe this is because it was also using a vocab size of 32K?

**Q4.** Suppose we know that the acronym “FY” will likely appear very frequently in most of the input text (assume the text comes from the financial domain). Therefore, we hope that adding it manually to the vocabulary might help. Add the token “FY” to the vocabulary and tokenize the input text. Enter the number of tokens produced.

[Question to ponder: Does reducing the number of tokens helpful?]

In [63]:
tokenizer_with_fy = Tokenizer(BPE())
tokenizer_with_fy = tokenizer_with_fy.from_file("hopper.json")

# manually adding the token "FY" to the vocab
tokenizer_with_fy.add_tokens(["FY"])

tokenizer_with_fy.encode(input_text)

Encoding(num_tokens=22, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

Now, the number of tokens required has reduced to **22**.

In [64]:
pprint(tokenizer_with_fy.encode(input_text).tokens, compact=True)

['seb', '##i', 'study', 'finds', '9', '##3', '%', 'of', 'individual', 'f', '&',
 'o', 'traders', 'made', 'losses', 'between', 'fy', '22', 'and', 'fy', '24',
 '.']


**Q5.** Load the “bert-base-uncased” and "gpt2” tokenizers (use AutoTokenizer function from transformers). Which of the following special tokens are used in these tokenizers? (MSQ)


- [ ] [GO]
- [x] [CLS]
- [ ] [BOS]
- [x] [SEP]
- [x] [< |endoftext| >]


In [74]:
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_tokenizer.special_tokens_map

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [78]:
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt2_tokenizer.special_tokens_map

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

**Q6.** By now, we have four tokenizers.

1. **Custom tokenizer** (vocab size 32K, trained on 10 million samples) : stored in `tokenizer_32k` in my case
2. **bert-base-uncased** : stored in `bert_tokenizer` in my case
3. **gpt2** : stored in `gpt2_tokenizer` in my case
4. **hopper** :  stored in `trained_tokenizer` here

Use these four tokenizers to count the number of tokens for the entire “imdb” dataset (drop the “unsupervised” part of the dataset). Enter the tokenizers in order such that the size of the dataset (measured in tokens) as returned by the tokenizers is in increasing order. For example, if the first tokenizer yields the smallest number of tokens and the fourth tokenizer yields the largest, you would enter 1234 (without any spaces).”

In [85]:
from datasets import get_dataset_infos, get_dataset_split_names

In [86]:
get_dataset_split_names("stanfordnlp/imdb")

['train', 'test', 'unsupervised']

In [93]:
imdb_ds = load_dataset("stanfordnlp/imdb", split="train+test")

In [112]:
imdb_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})

In [137]:
def count_tokens(tokenizer):
    # count the number of tokens in the imdb dataset using a specific tokenizer
    count = 0
    for sample in imdb_ds:
        count += len(tokenizer.encode(sample["text"]))
    return count

In [138]:
tokenizers = [("1. Custom Tokenizer", tokenizer_32k),
                ("2. bert-base-uncased", bert_tokenizer),
                ("3. gpt2", gpt2_tokenizer),
                ("4. hopper", trained_tokenizer)]

for tokenizer in tokenizers:
    print(f"Tokenizer: {tokenizer[0]}, Total Token Count: {count_tokens(tokenizer[1])}")

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Tokenizer: 1. Custom Tokenizer, Total Token Count: 15352840


Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


Tokenizer: 2. bert-base-uncased, Total Token Count: 15516058
Tokenizer: 3. gpt2, Total Token Count: 14812432
Tokenizer: 4. hopper, Total Token Count: 13526933


So, the hopper tokenizer (#4) uses the least amount of tokens, then comes the gpt tokenizer (#3) and then the custom tokenizer (#1), and finally bert tokenizer (#2).

So the answer is: **4312**

**Q7.** The statement that the special tokens and their respective token ids are model-specific (model here refers to a language model) is

- [x] True
- [ ] False

**Q8.** Suppose that the context length of the model is 128. Assume that a mini-batch of size 8 samples is passed to a tokenizer that corresponds to a model from hub. After tokenization, the maximum length of sample in the batch is 64. The statement that zero is appended to the “input ids” of the remaining samples to make the length 64 is :

- [ ] True
- [ ] False
- [x] Insufficient information to conclude


We don't know whether padding is enabled or not, so its hard to say whether the other samples will get padded to make their length 64
